#####  Method1 : using embeddings and Clusterings 
Imagine a platform that processes a continuous stream of conversations from a 
generative agent (StoryBot). To maintain a safe and engaging environment, you need to monitor these conversations in real time for anomalies such as sudden shifts in sentiment, unusual topic spikes, or atypical user behavior that could indicate emerging issues or opportunities. 
**Task**
-  Develop a streaming machine learning pipeline in Python that ingests data (message 
and metadata), does feature extraction and preprocessing in near real time.
-   applies an anomaly detection model (your own or from a 3rd party), and returns alerts when anomalies are detected. 
- For this task, feel free to pick an anomaly you think would be interesting to track (e.g., changes in mood, change in emoji use, shifts in language, use of the phrase “I don’t know”,  prompt injection attacks, etc.).
- Provide a brief README that outlines the architecture, setup instructions, and how to run tests.   
The datasets include:

**Conversations** - One-on-one conversations between users and an AI agent (StoryBot)
### conversations.json

Contains conversations between individual users and StoryBot. Each conversation includes:

- `messages_list`: List of messages in the conversation
- `ref_conversation_id`: Unique identifier for the conversation
- `ref_user_id`: ID of the user participating in the conversation

Example structure:

```json
{
  "messages_list": [
    {"message": "Good morning! How are you feeling today?",
    "ref_conversation_id": 42615,
    "ref_user_id": 1,
    "transaction_datetime_utc":  "2023-10-01T08:00:00Z",
    "screen_name": "StoryBot"
    },
    {"message": "I'm doing well, thanks for asking! Just trying to get through the day.",
    "ref_conversation_id": 42615,
    "ref_user_id": 822,
    "transaction_datetime_utc":  "2023-10-01T08:01:00Z",
    "screen_name": "User822"
    },
    {"message": "That's great to hear! Is there anything specific on your mind?",
    "ref_conversation_id": 42615,
    "ref_user_id": 1,
    "transaction_datetime_utc":  "2023-10-01T08:02:00Z",
    "screen_name": "StoryBot"
    },
  ],
  "ref_conversation_id": 42615,
  "ref_user_id": 822
}
```

##### Import Required Libraries

In [1]:
#import libarries
import json
import numpy as np
import pandas as pd
from datetime import datetime
import os
import matplotlib.pyplot as plt

##### Read JSON File to Load Conversations

- Define the folder and file paths for the dataset.
- Prepare to load the `conversations.json` file from the `data` directory within the current working directory.



In [2]:
#get folder and filepaths
#
curr_folder = os.getcwd()
conver_json = "conversations.json"
json_file = os.path.join(curr_folder,"data",conver_json)

##### Load Conversations from JSON

- Define a function `read_json()` to load conversation data from the specified JSON file.
    - Handles file not found and JSON decoding errors gracefully.
- Read the raw conversation data into memory.
- Count and print the total number of conversations loaded.


In [ ]:
def read_json(file_path):
    """
    Loads conversations from the conversations.json file.
    Args:
        file_path (str): The path to the JSON file.
    Returns:
        list: A list of conversation objects, or None if an error occurs.
    """
    try:
        with open(json_file, 'r') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Error: File not found: {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from the file {file_path}.")
        return None
# read RAW JSON data
raw_conversations = read_json(json_file)
total_conv = len(raw_conversations)
print(f"number of conversations is {total_conv}")


##### Split Data into Training and Test Sets

- Use `train_test_split` from scikit-learn to divide the loaded conversations into training and test sets.
- Set a random seed for reproducibility.
- Define the percentage split between training and testing (default: 80% train, 20% test).
- If the dataset is very small, warn the user and suggest using a K-fold strategy.
- Print the number of conversations in each split.  

**NOTE**: Alternatively, this could all be training set, and we test on a different set of conversations

In [ ]:
from sklearn.model_selection import train_test_split # For splitting data
import random

# Assuming StoryBot's ID is 1, based on given json file 
STORYBOT_USER_ID = 1 

RANDOM_SEED = 42 # For reproducible splits
train_per = 80
test_per = 100 - train_per
min_conv_to_train = 8
if total_conv > 0:
    # Too small a data to train
    if total_conv < min_conv_to_train: 
        print("Warning: Very few conversations. Use K-fold strategy to train and test")        
    else:
        train_conversations, test_conversations = train_test_split(
            raw_conversations, 
            test_size= test_per/100,       
            random_state=RANDOM_SEED # For reproducibility
        )

    print(f"\nNumber of conversations for training: {len(train_conversations)}")
    print(f"Number of conversations for testing: {len(test_conversations)}")   
else:
    print("Cannot proceed with splitting as no conversations were loaded.")
    train_conversations, test_conversations = [], [] # Initialize as empty

##### Define Function to Extract User Message Details

- Define `extract_user_message_details()` to:
    - Iterate through each conversation and its messages.
    - Extract details for each message not sent by StoryBot, including:
        - Conversation ID
        - User ID
        - Screen name
        - Timestamp (parsed as datetime)
        - Message text
    - Handle malformed message data gracefully.
- Returns a list of dictionaries, each representing a user message with relevant metadata.


In [5]:
def extract_user_message_details(list_of_conversations, is_storybot_user_id):
    """
    Extracts message details (text, conv_id, timestamp) for all messages for all users
    NOT sent by the is_storybot_user_id.

    Args:
        list_of_conversations (list): A list of conversation objects.
        is_storybot_user_id (int): The user ID of StoryBot.

    Returns:
        list: A list of dictionaries, where each dict contains 
              'conversation_id', 'original_user_id', 'screen_name', 
              'timestamp', and 'message_text' for a user message.
    """
    user_messages_details = []
    # for each conversation with  a user
    for conversation in list_of_conversations:
        # exgtract conversation id
        conv_id = conversation.get('ref_conversation_id', 'unknown_conv_id')
        # extract list of  messages
        messages_list = conversation.get('messages_list', [])
        # build new list with updated datetime
        for message_data in messages_list:
            if not isinstance(message_data, dict): # Basic check for message structure
                # print(f"Skipping malformed message in conv {conv_id}")
                continue

            message_user_id = message_data.get('ref_user_id')
            
            if message_user_id is not None and message_user_id != is_storybot_user_id:
                try:
                    timestamp_str = message_data.get('transaction_datetime_utc')
                    parsed_timestamp = None
                    if timestamp_str:
                        parsed_timestamp = datetime.fromisoformat(timestamp_str.replace('Z', '+00:00'))
                    
                    user_messages_details.append({
                        'conversation_id': conv_id,
                        'original_user_id': message_user_id,
                        'screen_name': message_data.get('screen_name', 'UnknownUser'),
                        'timestamp': parsed_timestamp,
                        'message_text': message_data.get('message', '') # Ensure text is a string
                    })
                except Exception as e:
                    print(f"Error processing a user message in conv {conv_id}: {e}. Message data: {message_data}")

    return user_messages_details


##### Extract Message Data for Training and Test

- Use the previously defined function to extract user messages from both training and test sets.
- Print the number of user messages extracted for each set.
- Display an example message for sanity checking.
- These extracted messages will be used for embedding and downstream anomaly detection.

In [ ]:

# Extract user messages for training
train_user_message_details = []
if train_conversations:
    # create list of all training user data , with updated time
    train_user_message_details = extract_user_message_details(train_conversations, STORYBOT_USER_ID)
    print(f"\nExtracted {len(train_user_message_details)} user messages for training.")
    # print first  user message for sanity check
    if train_user_message_details:
        print(f"Example training user message detail: {train_user_message_details[0]}")

# create list of all test user data , with updated time
test_user_message_details = []
if test_conversations:
    test_user_message_details = extract_user_message_details(test_conversations, STORYBOT_USER_ID)
    print(f"Extracted {len(test_user_message_details)} user messages for testing.")
    if test_user_message_details:
        print(f"Example testing user message detail: {test_user_message_details[0]}")

# exytract  'message_text' from these lists.

train_user_texts = [detail['message_text'] for detail in train_user_message_details]
test_user_texts = [detail['message_text'] for detail in test_user_message_details]




#### create embeddings: which model to use
-  Sentence Transformers :specifically designed to make generating high-quality sentence and text embeddings very straightforward
-  Hugging Face Transformers : massive number of pre-trained Transformer models like BERT, RoBERTa, DistilBERT (need to handle tokenization and pooling myself)
So, decided with Sentence tranformer.
Now, I have various models to choose based on :  
1   Size  
2   Speed  
3   Quality   
4   paraphrasing, semantic similaity sensitivity  
Starting with `all-MiniLM-L6-v2`, but having options for `all-mpnet-base-v2` and `paraphrase-MiniLM-L6-v2`

##### Define Function to Create Embeddings

- Define `create_embeddings()` to generate embeddings for a list of texts using a pre-loaded Sentence Transformer model.
- Handles empty input gracefully.
- Returns a NumPy array containing the embeddings.
- Prints the number of sentences encoded and the resulting embedding shape.

**NOTE** This approach is called **message-level anomaly detection**. I  decided on this because with only 24 total conversations (19 for training), trying to create a single embedding for an entire conversation (and then clustering only 19 such conversation-level embeddings) would likely be less stable and less effective than working with the larger number of individual user messages.

In [7]:
from sentence_transformers import SentenceTransformer

def create_embeddings(texts_to_encode, loaded_model):
    """
    Calculates embeddings for a list of texts using a pre-loaded Sentence Transformer model.

    Args:
        texts_to_encode (list of str): The list of sentences to encode.
        loaded_model (SentenceTransformer): The pre-loaded Sentence Transformer model.        
    Returns:
        numpy.ndarray: A NumPy array containing the embeddings.
    """
    if not texts_to_encode: # Handle empty input
        print("Input sentences list is empty. Returning an empty array.")        
        return np.array([]).reshape(0, loaded_model.get_sentence_embedding_dimension() if hasattr(loaded_model, 'get_sentence_embedding_dimension') else 384) # Assuming 384 for MiniLM if not found

    print(f"Encoding {len(texts_to_encode)} sentences...")
    embeddings = loaded_model.encode(texts_to_encode)
    print(f"Generated embeddings with shape: {embeddings.shape}")
    return embeddings



##### Generate Embeddings for Training and Test Messages

- Load the selected Sentence Transformer model.
- Generate embeddings for both training and test user messages.
- Print confirmation messages and embedding shapes for verification.


In [ ]:
#### call embedding creation
sen_Tran_model_names = ["all-MiniLM-L6-v2","all-mpnet-base-v2","paraphrase-MiniLM-L6-v2"]
# Load the model ONCE
selected_model_name = sen_Tran_model_names[0]
print(f"Loading Sentence Transformer model: {selected_model_name}....")
sentence_model = SentenceTransformer(selected_model_name)
print("Model loaded Sucessffuly.")
# training embeddings
tr_embed = create_embeddings(train_user_texts,sentence_model)
# test  embeddings
tst_embed = create_embeddings(test_user_texts,sentence_model)

##### Dimensionality Reduction: Why Use PCA?

- Considered using UMAP and KMeans for dimensionality reduction and clustering, but decided against UMAP because:
    - UMAP may distort distances or remove subtle semantic features important for anomaly detection.
    - Anomalies found in a lower-dimensional space may not correspond to real anomalies in the original embedding space.
- Instead, use PCA for dimensionality reduction to retain as much variance as possible while reducing dimensions.

##### Apply PCA to Embeddings

- Use PCA to reduce the dimensionality of the message embeddings.
    - Retain 90% of the variance (or set a specific number of components).
- Standardize embeddings before applying PCA for better performance.
- Fit PCA on the training embeddings and transform both training and test embeddings.
- Print explained variance ratio, number of components selected, and new embedding shapes.
- Warn if there is a dimension mismatch between train and test embeddings.


In [ ]:
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
 
# Choose 80% of the variance.
n_components_pca = 0.9
random_state_pca = 42 # For reproducibility

print(f"\nOriginal training embedding shape: {tr_embed.shape}")

# Initialize PCA
# If n_components is a float (0 to 1.0), it's the variance ratio to keep.
# If it's an int, it's the number of components.
pca_reducer = PCA(
    n_components=n_components_pca,
    random_state=random_state_pca
)
print(f"Fitting PCA on training embeddings to retain {n_components_pca*100 if isinstance(n_components_pca, float) else n_components_pca} components/variance...")

# AppLy standard scaler first to training embeddings
scaler = StandardScaler()
scaled_tr_embed = scaler.fit_transform(tr_embed)
# apply same scaling to Test embeddings
scaled_tst_embed = scaler.transform(tst_embed) 



# apply PCA on train embeddings
pca_tr_embeddings = pca_reducer.fit_transform(scaled_tr_embed)
print(f"Explained variance ratio by chosen components: {np.sum(pca_reducer.explained_variance_ratio_):.4f}")
print(f"Number of components selected by PCA: {pca_reducer.n_components_}")
print(f"Reduced training embedding shape: {pca_tr_embeddings.shape}")

# Transform the test embeddings using the *fitted* PCA reducer
pca_tst_embeddings = None
if tst_embed.shape[1] == tr_embed.shape[1]: 
    pca_tst_embeddings = pca_reducer.transform(scaled_tst_embed)
    print(f"Reduced test embedding shape: {pca_tst_embeddings.shape}")
else:
    print(f"Warning: Test embeddings feature dimension ({tst_embed.shape[1]}) does not match training ({tr_embed.shape[1]}). Skipping PCA transform for test data.")
    pca_tst_embeddings = np.array([]) # Ensure it's an empty array for later checks



##### Create K-Means Clusters for Training Embeddings

- Use K-Means clustering to group similar message embeddings in the reduced PCA space.
- Test a range of cluster counts (k) from 2 to 15.
- For each k:
    - Fit K-Means and assign cluster labels.
    - Calculate Within-Cluster Sum of Squares (WCSS) for the Elbow Method.
    - Calculate the Silhouette Score for clustering quality.
- Plot WCSS and Silhouette Scores to help select the optimal number of clusters.
- Choose the optimal k based on these plots and print the choice.


In [ ]:
from sklearn.metrics import silhouette_score 
k_range = range(2, 16) 
wcss_scores = []
silhouette_scores = []
random_state_kmeans = 42 
for k_val in k_range:
    kmeans_temp = KMeans(
        n_clusters=k_val,
        random_state=random_state_kmeans,
        n_init=10  # Explicitly set n_init to avoid warnings and ensure multiple runs
    )
    cluster_labels_temp = kmeans_temp.fit_predict(pca_tr_embeddings)
    
    # 1. WCSS (Inertia) for Elbow Method
    wcss_scores.append(kmeans_temp.inertia_)
    
    # 2. Silhouette Score
    #    Ensure there's more than 1 unique label to calculate silhouette score
    if len(np.unique(cluster_labels_temp)) > 1:
        score = silhouette_score(pca_tr_embeddings, cluster_labels_temp)
        silhouette_scores.append(score)
    else:
        silhouette_scores.append(-1) # Or some other indicator of invalid score for this k
    
    print(f"  For k={k_val}, WCSS: {wcss_scores[-1]:.2f}, Silhouette Score: {silhouette_scores[-1]:.4f}")

# Plot Elbow Method (WCSS)
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(k_range, wcss_scores, marker='o', linestyle='--')
plt.title('Elbow Method for Optimal k (WCSS)')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('WCSS (Inertia)')
plt.xticks(list(k_range))
plt.grid(True)

# Plot Silhouette Scores
plt.subplot(1, 2, 2)
plt.plot(k_range, silhouette_scores, marker='o', linestyle='--')
plt.title('Silhouette Scores for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Average Silhouette Score')
plt.xticks(list(k_range))
plt.grid(True)

plt.tight_layout()
plt.show()

print("\n--- Guidance for Choosing k ---")
print("Elbow Method: Look for a point on the WCSS plot where the rate of decrease sharply changes (the 'elbow').")
print("Silhouette Score: Look for the k that gives the highest average Silhouette Score.")
optimal_k = 11
print(f"CHOOSING Optimal K={optimal_k}")


##### Fit K-Means and Predict Clusters for Test Embeddings

- Initialize K-Means with the chosen optimal number of clusters.
- Fit K-Means on the PCA-reduced training embeddings and assign cluster labels.
- Obtain cluster centroids in the PCA-reduced space.
- Predict cluster labels for the PCA-reduced test embeddings.
- Print confirmation messages and handle dimension mismatches gracefully.


In [ ]:
# Initialize K-Means with the chosen optimal_k
kmeans_model = KMeans(
    n_clusters=optimal_k, # Use the k you determined from the plots
    random_state=random_state_kmeans,
    n_init=10
)

# Fit K-Means on the PCA-reduced training embeddings and get cluster labels for training data
train_cluster_labels = kmeans_model.fit_predict(pca_tr_embeddings)
print(f"K-Means fitting complete with k={optimal_k}. Cluster labels assigned to training data.")
# Get the cluster centroids (these are in the PCA-reduced space)
cluster_centroids = kmeans_model.cluster_centers_
print(f"Shape of cluster centroids: {cluster_centroids.shape}") # Should be (optimal_k, n_components_from_PCA)

# Predict cluster labels for the PCA-reduced test embeddings
test_cluster_labels = np.array([]) # Initialize as empty
if pca_tst_embeddings is not None and pca_tst_embeddings.size > 0:
    if pca_tst_embeddings.shape[1] == pca_tr_embeddings.shape[1]: # Check if dimensions match
        test_cluster_labels = kmeans_model.predict(pca_tst_embeddings)
        print(f"Cluster labels predicted for PCA-reduced test data.")       
    else:
        print(f"Warning: Dimension mismatch between PCA-reduced train ({pca_tr_embeddings.shape[1]}) and test ({pca_tst_embeddings.shape[1]}) embeddings. Cannot predict test labels.")
else:
    print("No PCA-reduced test embeddings to predict clusters for")


##### Anomaly Threshold for Clustering

- Define an anomaly threshold based on the distribution of distances from training points to their nearest cluster centroid.
- Use the 95th percentile of these distances as the anomaly threshold.
    - This means the top 5% most distant training points define what is considered "anomalous."
    - Note: Being in the top 5% does not necessarily mean a message is "bad"-it could simply be rare or unique.
- For each test point, calculate its minimum distance to any cluster centroid.
- Flag test points as anomalies if their distance exceeds the threshold.
- Print the number of anomalies detected in the test set.


In [ ]:
# --- Anomaly Detection on Test Set using Distance to Nearest KMeans Centroid ---
anom_Threhold = 99 /100
 # Will hold min distance of each test point to any centroid
min_distances_test = np.array([])   
        

# : Compute distances for each test point to all centroids 
if pca_tst_embeddings is not None and pca_tst_embeddings.size > 0 and cluster_centroids.size > 0:
    if pca_tst_embeddings.shape[1] == cluster_centroids.shape[1]:
        distances_to_all_centroids_test = kmeans_model.transform(pca_tst_embeddings)  # shape: (num_test_samples, num_clusters)
        min_distances_test = np.min(distances_to_all_centroids_test, axis=1)         # min distance per test point
        

#  Compute threshold using training data distance
if pca_tr_embeddings.size > 0 and cluster_centroids.size > 0:
    distances_to_all_centroids_train = kmeans_model.transform(pca_tr_embeddings)
    min_distances_train = np.min(distances_to_all_centroids_train, axis=1)

    if min_distances_train.size > 0:
        # Choose the 95th percentile of training distances as anomaly threshold
        anomaly_distance_threshold = np.percentile(min_distances_train, anom_Threhold)
        print(f"Calculated anomaly distance threshold (95th percentile of train distances): {anomaly_distance_threshold:.4f}")
        #Flag test points as anomalies if distance > threshold 
        if min_distances_test.size > 0 and anomaly_distance_threshold is not None:
            test_anomalies_flags = min_distances_test > anomaly_distance_threshold
            if test_anomalies_flags.size > 0:                
                print(f"Number of anomalies detected in test set: {np.sum(test_anomalies_flags)} out of {len(test_anomalies_flags)}")
# for i, is_anomalous_flag in enumerate(test_anomalies_flags):
#     if is_anomalous_flag:
#     # Ensure the message detail has 'conversation_id'
#         if i < len(test_user_message_details) and 'conversation_id' in test_user_message_details[i]:
#             anomalous_message_conv_ids.add(test_user_message_details[i]['conversation_id'])
#         else:
#             print(f"Warning: Missing 'conversation_id' or index out of bounds for anomalous message at index {i} in test_user_message_details.")